In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE98697"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE98697"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE98697.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE98697.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE98697.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Coding and noncoding gene expression of 48 pediatric AML samples"
!Series_summary	"Long non-coding RNAs (lncRNAs) and miRNAs have emerged as crucial regulators of gene expression and cell fate decisions. Here we present an integrated analysis of the ncRNA-transcriptome of purified human hematopoietic stem cells (HSCs) and their differentiated progenies, including granulocytes, monocytes, T-cells, NK-cells, B-cells, megakaryocytes and erythroid precursors, which we correlated with the ncRNA expression profile of 48 pediatric AML samples to establish a core lncRNA stem cell signature in AML.Linear (PCA) and nonlinear (t-SNE) dimensionality reduction of 46 pediatric AML samples including Down syndrome AMKL, core-binding factor AMLs (inv[16] or t[8;21]) and MLL-rearranged leukemias mapped most samples to a space between HSCs and differentiated cells together with the myeloid progenitors. A subset of AML-samples mapped closely to healthy HSCs, includin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Dict, Any, Callable, Optional
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray data of coding and noncoding gene expression
# The "Series_overall_design" mentions "Microarray Analysis on the Arraystar Human LncRNA microarray" platform
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: Looking at the sample characteristics, row 2 contains AML subtypes
# This is not intellectual disability data, but AML (Acute Myeloid Leukemia) data
# Therefore, the intellectual disability trait is not available in this dataset
trait_row = None  # No intellectual disability data

# For age: There's no specific age mentioned in the sample characteristics
age_row = None

# For gender: There's no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
# Define conversion functions even though they might not be used in this dataset

def convert_trait(value: str) -> Optional[int]:
    """Convert trait value to binary (0 or 1)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Since this dataset doesn't have intellectual disability data, 
    # we would return None for all values
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous (float)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort information for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 62738 rows
First 20 gene IDs:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous output
# The identifiers appear to be simple numeric values (1, 2, 3, etc.)
# These are not standard human gene symbols which typically look like:
# BRCA1, TP53, APOE, etc.

# These numeric identifiers will need to be mapped to standard gene symbols
# for meaningful analysis and interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3074210 rows

Gene annotation preview (first few rows):
{'ID': ['1', '2', '3', '4', '5'], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'ASHG19A3A019218', 'ASHG19A3A052390'], 'ORF': [nan, nan, nan, 'DOPEY2', 'SCAMP2'], 'FINAL_SYMBOL': [nan, nan, nan, 'DOPEY2', 'SCAMP2'], 'FINAL_SOURCE': [nan, nan, nan, 'Gencode.V23', 'Gencode.V23'], 'FINAL_TYPE': [nan, nan, nan, 'coding', 'coding'], 'Ensembl.Symbol': [nan, nan, nan, 'DOPEY2', 'SCAMP2'], 'Ensembl.Gene': [nan, nan, nan, 'ENSG00000142197.12', 'ENSG00000140497.16'], 'Ensembl.Type': [nan, nan, nan, 'protein_coding', 'protein_coding'], 'chrom': [nan, nan, nan, 'chr21', 'chr15'], 'txEnd': [nan, nan, nan, 37666571.0, 75165670.0], 'txStart': [nan, nan, nan, 37536838.0, 75137196.0], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'ASHG19A3A019218', 'ASHG19A3A052390']}

Column names in gene annotation data:
['ID', 'NAME', 'ORF', 'FINAL_SYMBOL', 'FINAL_SOURCE', 'FINAL_TYPE', 'Ensembl.Symbol',

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for mapping
# Based on the annotation preview, we need to map:
# - 'ID' column from gene_annotation (numeric identifiers like '1', '2', '3') to the gene expression data index
# - 'FINAL_SYMBOL' column contains gene symbols (like 'DOPEY2', 'SCAMP2')

print("Creating gene mapping from ID to gene symbols...")

# 2. Create gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='FINAL_SYMBOL')

# Print some information about the mapping
print(f"Created mapping with {len(mapping_data)} entries")
print("Sample of mapping data:")
print(mapping_data.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe measurements to gene expression data...")
try:
    gene_data = apply_gene_mapping(gene_data, mapping_data)
    print(f"Successfully mapped probes to {len(gene_data)} genes")
    print("First few genes in the mapped data:")
    print(gene_data.index[:10])
    
    # Normalize gene symbols to ensure consistency
    print("\nNormalizing gene symbols...")
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data)} genes")
    
    # Save the processed gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Saved processed gene data to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")


Creating gene mapping from ID to gene symbols...
Created mapping with 61332 entries
Sample of mapping data:
  ID             Gene
3  4           DOPEY2
4  5           SCAMP2
5  6           uc.387
6  7  lincRNA-SPATS2L
7  8             VIL1

Converting probe measurements to gene expression data...


Successfully mapped probes to 37220 genes
First few genes in the mapped data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT', 'AA155639'],
      dtype='object', name='Gene')

Normalizing gene symbols...
After normalization: 24360 genes


Saved processed gene data to ../../output/preprocess/Intellectual_Disability/gene_data/GSE98697.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the expression data...")
try:
    # If previous steps have already loaded gene_data
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Save normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Link clinical and genetic data
if is_gene_available and 'clinical_df' in locals():
    print("\nLinking clinical and genetic data...")
    try:
        # Transpose normalized gene data to match clinical data format
        gene_data_t = normalized_gene_data.T
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_t)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 3. Handle missing values systematically
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"After handling missing values, data shape: {linked_data.shape}")
        
        # 4. Determine whether features are biased
        print("\nChecking for bias in features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        
        # 5. Final validation and save metadata
        print("\nPerforming final validation...")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=is_biased,
            df=linked_data,
            note=""
        )
        
        # 6. Save the linked data if usable
        if is_usable:
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            
            # Save linked data
            linked_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print(f"Dataset not usable for {trait} association studies. Data not saved.")
    
    except Exception as e:
        print(f"Error in data linking or processing: {e}")
        is_usable = False
else:
    print("Cannot link data: either gene data or clinical data is unavailable.")
    # Create a minimal dataframe for validation purposes
    import pandas as pd
    linked_data = pd.DataFrame({trait: [None]})
    
    # Perform final validation with appropriate flags
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Not relevant since data isn't usable
        df=linked_data,
        note="Failed to link gene and clinical data."
    )
    print(f"Dataset usability: {is_usable}")

Normalizing gene symbols in the expression data...


Normalized gene data saved to ../../output/preprocess/Intellectual_Disability/gene_data/GSE98697.csv
Normalized gene data shape: (24360, 48)
Cannot link data: either gene data or clinical data is unavailable.
Abnormality detected in the cohort: GSE98697. Preprocessing failed.
Dataset usability: False
